In [ ]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Weighted overlay
Here's a recipe for assigning a quantity from one set of polygons to another, based on the areas of overlap.

In [ ]:
sa2 = gpd.read_file("data/sa2-wellington.gpkg")
school_zones = gpd.read_file("data/school-zones.gpkg")
ax = sa2.plot(alpha = 0.5, ec = "w", lw = 0.5, figsize = (8, 8))
school_zones.plot(ax = ax, fc = "red", alpha = 0.5, ec = "k", lw = 0.5)

We need the areas of the SA2 polygons.

In [ ]:
sa2["sa2_area"] = sa2.area
sa2

Next, overlay the school zones and the SA2s, and calculate the areas of the intersections.

In [ ]:
school_zones_sa2 = school_zones.overlay(sa2)
school_zones_sa2["area"] = school_zones_sa2.area

region = gpd.GeoSeries([sa2.union_all().union(school_zones.union_all())])

Now we can estimate populations of the school zones, make a new dataframe with this information and merge it back into the school zones data. 

In [ ]:
school_zones_sa2["pop"] = \
    school_zones_sa2.CURPop * school_zones_sa2.area / school_zones_sa2.sa2_area
school_zone_pops = school_zones_sa2 \
    .groupby("School_ID", as_index = False) \
    .agg({"pop": "sum",
          "area": "sum"})
school_zones = school_zones.merge(school_zone_pops)
school_zones["pop_density"] = school_zones["pop"] / school_zones.area * 1e6

In [ ]:
ax = region.plot(fc = "#00000000", ec = "k", figsize = (8, 8))
school_zones.plot(ax = ax, column = "pop_density", cmap = "Reds", alpha = 0.5,
                  k = 9, scheme = "equalinterval", ec = "k", lw = 0.35, 
                  legend = True, legend_kwds = {"loc": "upper left"})
ax.set_axis_off()
plt.show()